# OpenCV로 동영상 만들기

1. `VideoWriter_fourcc`로 동영상 포맷 변수 생성하기
    - fourcc = cv2.VideoWriter_fourcc(*'mpv4')
    - fourcc = cv2.VideoWriter_fourcc(*'XVID')
2. `VideoWriter`로 동영상 생성하기
    - video = cv2.VideoWriter(제목, 포맷, 초당 프레임 수, 프레임사이즈(w, h))
3. `write`로 이미지 추가하기
    - video.write(이미지)
        - 이미지는 np array 타입
4. `release`로 완성하기
    - video.release()

In [14]:
import cv2
import time

In [40]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
#fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')

img = cv2.imread('book1.jpg')

video = cv2.VideoWriter("out.mp4", fourcc, 1.0, (img.shape[1], img.shape[0]))
#동영상 제목, 동영상포맷, 초당 프레임, 프레임 사이즈

for i in range(100):
    k = i + 1
    if i != 0:
        k = i // 25 + 1
    img = cv2.imread(f'book{k}.jpg')
    video.write(img)
    time.sleep(0.01)
    
video.release()

In [18]:
print(type(fourcc))
print(fourcc)

<class 'int'>
1145656920


## * : argument unpacking

In [5]:
def fun(a, b, c):
    print(a)
    print(b)
    print(c)
    
fun(1, 2, 3)

fun(*[1, 2, 3])

fun(*"123")

1
2
3
1
2
3
1
2
3


## Yolo 결과를 비디오로 만들어보자

In [51]:
import cv2 as cv
import argparse
import numpy as np
import os.path
from matplotlib import pyplot as plt
%matplotlib inline

# Initialize the parameters
confThreshold = 0.5  #Confidence threshold; 컨피던스가 이 값 미만이면 무시할 것임; 관행적으로 0.5를 쓴다.
nmsThreshold = 0.4   #Non-maximum suppression threshold; 주변보다 크거나 작은 값만 살림; 
inpWidth = 416       #Width of network's input image
inpHeight = 416      #Height of network's input image

# Load names of classes
classesFile = "coco.names"    #클래스명을 정리한 텍스트파일
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
print(classes)
# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "yolov3.cfg"     #네트워크를 구성하는 레이어들의 파라미터를 정리한 텍스트파일
modelWeights = "yolov3.weights"       #모델 학습 결과 도출된 weight값을 정리한 파일

net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)    #`readNetFromDarknet`: 외부 config 정보와 weight 정보로 네트워크 구축
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)    #백엔드 지정
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

# Get the names of the output layers;
#아웃풋 레이어명을 리턴하는 함수
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Draw the predicted bounding box;
#바운딩 박스 그리고 클래스 레이블/확률값 쓰는 함수
def drawPred(classId, conf, left, top, right, bottom):
    # Draw a bounding box.;
    #박스 그리기
    cv.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    
    label = '%.2f' % conf
        
    # Get the label for the class name and its confidence
    #클래스 레이블/확률값 구하기
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)

    #Display the label at the top of the bounding box;
    #클래스 레이블/확률값 쓸 위치 지정
    labelSize, baseLine = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv.FILLED)
    cv.putText(frame, label, (left, top), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)

# Remove the bounding boxes with low confidence using non-maxima suppression;
#
def postprocess(frame, outs):
    frameHeight = frame.shape[0]   #행 == 높이
    frameWidth = frame.shape[1]    #열 == 폭

    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:     #yolo layer 출력값을 1개씩 취함; 3번 돈다.
        for detection in out:     #바운딩박스 1개씩 취함; 507, 2028, 8112번 돈다.
            scores = detection[5:]    #80개 클래스에 대한 확률값
            classId = np.argmax(scores)     #가장 높은 확률값의 인덱스
            confidence = scores[classId]    #가장 높은 확률
            if confidence > confThreshold:    #확률이 미리 설정해둔 파라미터보다 높을 경우에만 실행된다.; confidence만으로도 많이 걸러짐
                center_x = int(detection[0] * frameWidth)   #detection 값이 0~1로 정규화되기 때문에 원래 값으로 되돌리기 위해서 폭/높이를 곱해준다.
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
    
    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [52]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')

cap = cv2.VideoCapture('vtest.avi') 
_, frame = cap.read()

video = cv2.VideoWriter("out.mp4", fourcc, 20.0, (frame.shape[1], frame.shape[0]))
#동영상 제목, 동영상포맷, 초당 프레임, 프레임 사이즈

for i in range(400):
    _, frame = cap.read()
    blob = cv.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)
    net.setInput(blob)
    outs = net.forward(getOutputsNames(net)) 
    postprocess(frame, outs)
    video.write(frame)
#     time.sleep(0.01)      여기서는 sleep 걸어줄 필요가 없다: 어차피 루프 상단부가 오래 걸림
    
video.release()
